In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#output_file-to save the layout in file, show-display the layout , output_notebook-to configure the default output state  to generate the output in jupytor notebook.
from bokeh.io import output_file, show, output_notebook 
#ColumnDataSource makes selection of the column easier and Select is used to create drop down 
from bokeh.models import ColumnDataSource, Select
#Figure objects have many glyph methods that can be used to draw vectorized graphical glyphs. example of glyphs-circle, line, scattter etc. 
from bokeh.plotting import figure 
#To create intractive plot we need this to add callback method.
from bokeh.models import CustomJS 
#This is for creating layout
from bokeh.layouts import column
output_notebook() #create default state to generate the output

from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

HTML("""
<style>
p { text-align: justify; }
}
</style>
""")
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Authors 
- James Venzor ([LinkedIn](https://www.linkedin.com/in/james-venzor-094ab5169))
- Matthew Gartenhaus ([LinkedIn](https://www.linkedin.com/in/matthew-gartenhaus-665070160?challengeId=AQHWGhQgt7lvEwAAAXbe4vvf5WMsmKROb6WAS5kKx7TJGm5y4VvY3EfYKkqZHtXEfTwm-1Gj8Yh4TJy2ifPQSVAkID3f8ddPKw&submissionId=7adb23f3-7a12-5816-e6f9-a34fd90f4b4a))

All code for this submission can be found in this [GitHub repository](https://github.com/jvenzor23/DefensiveCoverageNet). 

# Table of Contents

* [1. Introduction](#1)
* [2. Methods](#2)
    - [2.1. Classifying Coverage Type and Assignment](#2.1)
    - [2.2. Real-Time EPA Predictions](#2.2)
    - [2.3. Evalutation of Pass Coverage Attributes](#2.3)
        - [2.3.a. Tracking Metrics](#2.3.a)
        - [2.3.b. Closing, Ball Skills, Tackling, and INT Return Metrics](#2.3.b)
        - [2.3.c. Ball Hawk Metrics](#2.3.c)
* [3. Pass Coverage Ratings](#3)
    - [3.1. Best Players Per Pass Coverage Attribute](#3.1)
    - [3.2 Dashboard](#3.2)
* [4. Conclusion](#4)
* [5. Appendix](#5)

# 1. Introduction <a id="1"></a>
    
A variety of skills are required of a defender in order to effectively defend against the pass. In this submission, we analyze the NFL's 2018 Next Gen Stats data to quantify the performance of individual defenders in terms of expected points saved (EPS) across the following set of pass coverage attributes:

__<ins>Defensive Pass Coverage Attributes</ins>__
* __Tracking__: ability to closely track a receiver before a pass attempt
* __Closing__: ability to break on a pass and close on a receiver between a pass attempt and its arrival
* __Ball Skills__: ability to make a play on the ball upon arrival of the pass
* __Tackling__: ability to limit yards after catch by a receiver after a completion
* __INT Return__: ability to maximize the yards of a return following an interception
* __Ball Hawk__: ability to make a play on a pass targeted to a receiver the defender was not assigned to

We provide these metrics for each coverage type (man/zone) employed by a defender. Our quantification of defender performance by pass coverage attribute can enable front offices and coaches to evaluate the pass coverage performance of a player at a more granular level than currently possible.

# 2. Methods <a id="2"></a>

In [Section 2.1](#2.1), we classify the coverage type (man/zone) of a pass defender by implementing an unsupervised clustering algorithm. Next, in [Section 2.2](#2.2), we produce real-time EPA predictions over the course of each play by training a series of deep learning models using player tracking derived features. Finally, in [Section 2.3](#2.3), we provide methods to quantify a defender's EPS across our pass coverage attributes using the real-time EPA predictions obtained in Section 2.2.

## 2.1. Classifying Coverage Type and Assignment <a id="2.1"></a>

### Coverage Type
To classify the coverage type employed by a defender, we utilize gaussian mixture models to identify clusters corresponding to man and zone coverage, adapted from the paper by [Dutta et al.](https://arxiv.org/pdf/1906.11373.pdf). We augment the pass-coverage specific features fed to the mixture model to generalize the algorithm to classify coverage type for all positions. Additionally, we fit separate mixture models to classify coverage type for plays with pass attempts and a sacks[<sup>1</sup>](#fn1)<a id="fn1r"></a>. 

### Coverage Assignments
In addition to determining the type of coverage employed by a defender, we also identify each defender's coverage assignment throughout the course of a play. If we identify a defender as playing man coverage, we assign them to the receiver whose motion most closely matches that of the defender over the course of the play[<sup>2</sup>](#fn2)<a id="fn2r"></a>. 

For zone coverage, a defender's coverage assignment can change throughout the course of a play! Hence, we say that for any time $t$ prior to the outcome of a pass attempt, defender $x$ is assigned to receiver $y$ at time $t$ if and only if:
* Defender $x$ is the closest defender to receiver $y$ at time $t$
*  Receiver $y$ is the closest offensive player to defender $x$ at time $t$

Two animated plays, complete with each defender's coverage type and assignment over time, are provided in appendix [A.1](#A.1)

## 2.2. Real-Time EPA Predictions <a id="2.2"></a>

In order to evaluate defender performance across different pass coverage attributes during a play, it is first necessary to predict the eventual EPA of the play at various key points during the course of the play. We focus on predicting EPA values at the following time groupings within a play (the targeted receiver is known at each time grouping):

__<ins>Time Groupings</ins>__
* __Time of Pass Attempt__: Identified by `event = pass_forward` in the tracking data.
* __Time of Pass Arrival__: Identified by `event = pass_arrived` in the tracking data.
* __Time of Pass Outcome__: Identified by `event = pass_outcome_caught` or `event = pass_outcome_interception` in the tracking data (incompletions are not modeled because this pass outcome results in the end of a play). 

An illustration of our formulation for predicting these real-time EPA values at each of the above time groupings[<sup>3</sup>](#fn3) is provided below (where CNN stands for convolutional neural network). The function we use to convert yards gained or lost to EPA in (3) below is from an open source implementation of an EPA model[<sup>4</sup>](#fn4)<a id="fn4r"></a> provided by the nflWAR package [here](https://github.com/ryurko/nflscrapR-models/blob/master/R/init_models/init_ep_fg_models.R).

![](https://raw.githubusercontent.com/jvenzor23/DefensiveCoverageNet/master/src/08_writeup/images/EpaModelingFlowDiagram8.png)

![](https://raw.githubusercontent.com/jvenzor23/DefensiveCoverageNet/master/src/08_writeup/images/ModelEQNs3.png)

Equation (2.2.1) uses the outputs of (1) - (3) above to compute the predicted EPA of a play prior to the result of a pass, and Equations (2.2.2)-(2.2.3) above use the outputs of (2)-(3) above to compute the predicted EPA of a play at the time of the pass result.

The pass coverage specific features we feed to the CNN models and the model structures we employ are inspired by the [winning solution](https://www.kaggle.com/c/nfl-big-data-bowl-2020/discussion/119400) to the 2019-2020 Big Data Bowl. The pass coverage specific features we derived are described in appendix [A.2](#A.2). Information on the structure of each CNN model we employ can be found in the [GitHub repository](https://github.com/jvenzor23/DefensiveCoverageNet) for this submission. For each model, we filter out plays from the training data set with fumbles, penalties that impact the outcome of the play, or other disqualifying characteristics[<sup>5</sup>](#fn5)<a id="fn5r"></a>. Additionally, we increase the number of available training plays by producing a mirrored version of each play in the training set. 

Summaries of each CNN model created for this submission are listed in the table below.

![](https://raw.githubusercontent.com/jvenzor23/DefensiveCoverageNet/master/src/08_writeup/images/TotalModelStats3.png)

Due to the lack of sample size for plays with an interception, we predicted the expected return yardage of an interception using Model 5 above, by flipping each intercepted play to be from the perspective of the defense rather than the offense.

Consider the example play shown below (the defense is shown in blue, and the targeted receiver in red). At each time stamp, we provide our EPA prediction, as well as other predicted variables. 

![](https://raw.githubusercontent.com/jvenzor23/DefensiveCoverageNet/master/src/08_writeup/images/ScoredEPAsEx2_3.png)

For this play, our real-time predictions enable us to uncover that the positive EPA of the play was earned after the catch.


### Predicted Pre-Pass EPA of Target
We can also predict the pre-pass EPA of a target to each eligible receiver at all times prior to a pass attempt by scoring these real-time data with our models trained at the time of a pass attempt. We begin predicting these pre-pass EPA values at 0.5 seconds after the time of a snap, to ensure the quarterback is able to throw the football at the times we are scoring. An example play scored with these pre-throw EPA predictions is provided below.

![](https://raw.githubusercontent.com/jvenzor23/DefensiveCoverageNet/master/src/08_writeup/images/TrackingEPAsEx4.gif)

## 2.3. Evaluation of Pass Coverage Attributes <a id="2.3"></a>

This section focuses on applying the EPA predictions from the previous section to evaluate defensive player skill for different pass coverage attributes. The figure below illustrates the portions of a play that correspond to each attribute we cover in this section.

![](https://raw.githubusercontent.com/jvenzor23/DefensiveCoverageNet/master/src/08_writeup/images/PassCoverageAttributeTimeline5.png)

### 2.3.a. Tracking Metrics <a id="2.3.a"></a>

A defender's tracking EPS quantifies the extent to which a defender alters the predicted EPA at the time of a pass attempt based on their ability to cover their assigned receiver prior to a throw. Due to the complexity of zone coverage schemes, we only produce tracking metrics for defenders in man coverage.

Three factors must be considered to fairly assign EPS values to defenders in terms of tracking ability:

1. __The predicted pre-pass EPA of a target is not equivalent to the predicted EPA of the eventual pass attempt:__ If a defender blankets a receiver, the quarterback has other receiving options to throw to, causing the EPA of the pass attempt to be greater (on average) than the EPA of the blanketed route. Conversely, if a defender loses a route, it is possible the quarterback does not see the receiver and throws elsewhere, making the EPA of the pass attempt less than the EPA of the open route. The figure below quantifies this phenomenon by discretizing route EPAs at the time of a pass attempt into buckets, and averaging over the predicted EPA of the pass attempt. This estimates the function $f$ such that $$\widehat{\text{EPA}}^{PassAttempt} = f(\text{EPA}^{Route})$$
![](https://raw.githubusercontent.com/jvenzor23/DefensiveCoverageNet/master/src/08_writeup/images/PlayerTrackingPhysics7.png)
2. __Teams generate pressure on the quarterback with varying effectiveness__. Our metric for tracking EPS accounts for this by weighting each pre-throw EPA prediction by the overall likelihood of a throw occurring at that time, $P(T = t)$.
3. __The average EPA prediction at the time of a pass attempt is positive__. Since plays in which a pass attempt occur exclude sacks, it is not surprising that the average EPA prediction at the time of a pass attempt is positive. The average EPA prediction of a play at the time of a pass attempt is found to be: 
$$ \overline{\text{EPA}}^{PassAttempt} = +0.23 $$

Synthesizing the above three points, we estimate a defender's tracking EPS as follows: 

For each defender $i$, let 
* $n_i$ be the number of routes defender $i$ defended in man coverage, 
* $n_{t, i}$ be the number of routes defender $i$ defended in man coverage that lasted at least $t$ seconds after the snap
* $\text{max}(t)_i$ be the longest time after the snap defender $i$ had to cover in man coverage. 
* $\text{EPA}_{t,j}^{Route}$ be the predicted EPA value of the $j$th route that defender $i$ was covering in man coverage $t$ seconds after the snap and before a pass attempt

Then we quantify tracking EPS per play according to equation (2.3.1) below, and the total tracking EPS, according to equation (2.3.2) below. 

![](https://raw.githubusercontent.com/jvenzor23/DefensiveCoverageNet/master/src/08_writeup/images/TrackingEQNs2.png)

We also adjust each player's tracking EPS based on EPA accrued through penalties such as defensive holding or illegal contact. Example plays with predicted pre-pass EPAs of targets are provided in appendix [A.3](#A.3)

### 2.3.b. Closing, Ball Skills, Tackling, and INT Return Metrics <a id="2.3.b"></a>

Next, we quantify the EPS by defenders resulting from closing, ball skills, tackling, and INT return attributes. We quantify each defender's EPS per attribute by analyzing the shifts in the predicted real-time  EPAs over segments of a play where a defender is assigned to the targeted receiver.

Mathematically, for each defender $i$, let:
* $N^{Closing}_i$, $N^{BallSkills}_i$, $N^{Tackling}_i$ be the sets of plays in which defender $i$ is assigned to the targeted receiver at the time of the pass arrival, outcome, and completion respectively
* $N^{INTreturns}_i$ be the set of plays in which defender $i$ makes an interception
* $\text{EPA}^{'PassAttempt}_{ij}, \text{EPA}^{PassArrival}_{ij}, \text{EPA}^{PassOutcome}_{ij}, $ and $ \text{EPA}^{Play}_{ij} $ be the predicted EPA at the time of the pass attempt[<sup>6</sup>](#fn6)<a id="fn6r"></a>, pass arrival, pass outcome, and play result of the $j$th play for defender $i$ respectively. 

Then: 

$$
\text{EPS}_i^{Closing} =  \sum_{j \in N^{Closing}_i}\Big(\text{EPA}_{ij}^{'PassAttempt} - \text{EPA}_{ij}^{PassArrival}\Big)\tag{2.3.3} $$

$$ \text{EPS}_i^{BallSkills} =  \sum_{j \in N^{BallSkills}_i}\Big(\text{EPA}^{PassArrival}_{ij} - \text{EPA}_{ij}^{PassOutcome}\Big)\tag{2.3.4} $$

$$ \text{EPS}_i^{Tackling} =  \sum_{j \in N^{Tackling}_i}\Big(\text{EPA}^{PassOutcome}_{ij} - \text{EPA}^{Play}_{ij}\Big)\tag{2.3.5} $$

$$ \text{EPS}_i^{INTreturns} =  \sum_{j \in N^{INTreturns}_i}\Big(\text{EPA}^{PassOutcome}_{ij} - \text{EPA}^{Play}_{ij}\Big)\tag{2.3.6} $$

We compute each of these metrics separately by coverage type, and identify pass interference penalties[<sup>7</sup>](#fn7)<a id="fn7r"></a> and factor these into each player's defensive grade based on the EPA of each penalty. Example plays for closing, ball skills, and tackling plays are provided in [A.4](#A.4), [A.5](#A.5), and [A.6](#A.6) respectively.

### 2.3.c. Ball Hawk Metrics <a id="2.3.c"></a>

There are instances when a defender reads the quarterback and earns a pass breakup or interception on a pass targeted to a receiver the defender was not covering. We classify this skill as a player's "ball hawk" ability. An example ball hawk interception is shown in [A.7](#A.7).

Mathematically, for each defender $i$, let:
* $N^{BallHawk}_i$ be the set of plays in which defender $i$ earns a pass breakup or interception when defender $i$ was not covering the targeted receiver at the time of the pass attempt or pass arrival.
* $\text{EPA}^{PassAttempt}_{ij}$ and $\text{EPA}^{PassOutcome}_{ij}$ be the predicted EPAs at the time of the pass attempt and outcome for the $j$th play in which defender $i$ was not assigned to the targeted receiver

Then: 

$$
\text{EPS}_i^{BallHawk} =  \sum_{j \in N^{BallHawk}_i}\Big(\text{EPA}_{ij}^{PassAttempt} - \text{EPA}_{ij}^{PassOutcome}\Big)\tag{2.3.7} $$

# 3. Pass Coverage Ratings <a id="3"></a>

## 3.1. Best Players Per Pass Coverage Attribute <a id="3.1"></a>

In this section, we provide a quick view of the leaders across each attribute detailed in section 2.3 based on player performance in the 2018 NFL season. Leaders for each attribute (by coverage type) are provided, as well as overall EPS leaders, with a player's overall EPS representing the sum of their EPS values for each attribute. We also provide a Strength-of-Schedule adjusted (SOS)[<sup>8</sup>](#fn8)<a id="fn8r"></a> overall EPS that adjusts for the difficulty of each defender's offensive assignments. 

*_Note: The below plot may require a few minutes (or a refresh!) to render._

In [ ]:
imagesDf = pd.DataFrame({'Attribute':['Overall','Overall By Coverage Type','Tracking', 'Closing',
                                      'Ball Skills','Tackling','Ball Hawk','INT Returns'], 
        'URL':['https://raw.githubusercontent.com/jvenzor23/DefensiveCoverageNet/master/src/08_writeup/images/OverallScores2.png',
               'https://raw.githubusercontent.com/jvenzor23/DefensiveCoverageNet/master/src/08_writeup/images/OverallScoresManZone2.png',
               'https://raw.githubusercontent.com/jvenzor23/DefensiveCoverageNet/master/src/08_writeup/images/TrackingScores2.png',
               'https://raw.githubusercontent.com/jvenzor23/DefensiveCoverageNet/master/src/08_writeup/images/ClosingScores.png',
               'https://raw.githubusercontent.com/jvenzor23/DefensiveCoverageNet/master/src/08_writeup/images/BallSkillsScores2.png',
               'https://raw.githubusercontent.com/jvenzor23/DefensiveCoverageNet/master/src/08_writeup/images/TacklingScores.png',
               'https://raw.githubusercontent.com/jvenzor23/DefensiveCoverageNet/master/src/08_writeup/images/BallHawkScores2.png',
               'https://raw.githubusercontent.com/jvenzor23/DefensiveCoverageNet/master/src/08_writeup/images/IntReturnScores.png']})


imagesDf2 = imagesDf[imagesDf['Attribute'] == 'Overall']
   

Overall = ColumnDataSource(data=imagesDf)
Curr=ColumnDataSource(data=imagesDf2)


#plot and the menu is linked with each other by this callback function
callback = CustomJS(args=dict(source=Overall, sc=Curr), code="""
var f = cb_obj.value
sc.data['Attribute']=[]
sc.data['URL']=[]
for(var i = 0; i <= source.get_length(); i++){
	if (source.data['Attribute'][i] == f){
		sc.data['Attribute'].push(source.data['Attribute'][i])
		sc.data['URL'].push(source.data['URL'][i])
	 }
}   
   
sc.change.emit();
""")
menu = Select(options=list(imagesDf['Attribute'].unique()),value='Overall', title = 'Pass Coverage Attribute')  # drop down menu
# p=figure(x_axis_label ='Year', y_axis_label = 'fertility') #creating figure object 
# p.circle(x='Year', y='fertility', color='green', source=Curr) # plotting the data using glyph circle
p = figure(x_range=(0,1), y_range=(0,1), plot_width=726, plot_height = 242)
p.image_url(url="URL", x=0, y=1, h=1, w=1, source = Curr)
p.axis.visible = False
menu.js_on_change('value', callback) # calling the function on change of selection
layout=column(menu, p) # creating the layout
output_file('pandas.html')
show(layout) # displaying the layout

## 3.2 Dashboard <a id="3.2"></a>

While the above plots only display the top 15 players in each category, we also deploy a dashboard through shinyapps.io that contains all pass coverage attribute scores for all players. You can access this dashboard [here](https://mgartenhaus.shinyapps.io/BigDataBowlPassDefense/).

# 4. Conclusion <a id="4"></a>

In this submission, we quantified the performance of individual defenders in terms of EPS across a set of pass coverage attributes. There are plenty of opportunities to improve upon this work (for example, devising a system for estimating tracking EPS when in zone coverage), but we believe this work builds a foundation for quantifying defender performance across a set of pass coverage specific attributes.

---
<span id="fn1"> <sup>1</sup> The cross-validated adjusted Rand Index values of the clusters for plays with a pass attempt and plays ending with a sack are 0.96 and 0.93 respectively, which are consistent with the results provided by Dutta et al. Additionally, cornerbacks, linebackers, and safeties are identified as playing 55%, 21%, and 14% of their respective snaps in man coverage according to these mixture model classifications. </span>

<span id="fn2"> <sup>2</sup>We define the similarity of motion between a receiver and defender mathematically using pass coverage specific variables such as separation distance, correlation of motion during the play, etc. </span>

<span id="fn3"> <sup>3</sup>Step 1 in the flow diagram is omitted when predicting the EPA of a play at the time of the pass outcome, since the pass outcome we would be trying to predict is already known in this case! </span>

<span id="fn4"> <sup>4</sup>Specifically, we fit a multinomial log-linear model via neural networks to predict the EPA of a play based on down, yards to go, spot of the football, time remaining in half, and various other contextual variables. </span>

<span id="fn5"> <sup>5</sup>An example of a less obvious filtering criterion is that for Model 5, we exclude plays from the training set where a catch was made in the endzone for a touchdown, because the play is over immediately upon the catch, so there is no opportunity for YAC! </span>

<span id="fn6"> <sup>6</sup>There are some incompletions such as throw-aways, tipped passes, or inaccurate passes are never tagged with a pass_arrived event. As a result, the predicted EPAs of pass attempts for plays with a pass_arrived flag are under-estimated (since at the time of the pass attempt it was possible the pass would be inaccurate and not have a pass_arrived flag). To correct for this, we fit a linear regression to predict the  $\text{EPA}^{PassAttempt}$ value of a play from the $\text{EPA}^{PassArrived}$ value of the play, and $\text{EPA}^{'PassAttempt}$ represents this fitted value. </span>

<span id="fn7"> <sup>7</sup>We use the player tracking data to identify if a pass interference penalty occurs prior to the arrival of a pass or after the arrival. This enables us to classify some DPI penalties as occurring in the closing phase, and others (the majority) in the ball skills phase. </span>

<span id="fn8"> <sup>8</sup>Just as we compute EPS values by defender for each pass coverage attribute, we can equivalently compute EPA metrics by receiver (for example, we found Tyreek Hill added the most points in the tracking phase, and Julio Jones added the most points in the ball skills phase). We then baselined each defender's EPS scores with their expected score based on who they matched up with. This way, defenders that shadow #1 receivers get a boost in rating, since they had to cover more difficult opponents! </span>

# Appendix <a id="5"></a>
### A.1 Coverage Example <a id="A.1"></a>

Animations of two separate plays tagged with our coverage classifications and assignments are provided below. For each animation:
* a black line linking a receiver and defender denotes man coverage assignment
* a white outline around a defender indicates zone coverage classification
* a dashed line linking a receiver and defender indicates zone coverage assignment. 
* a red outline around a defender indicates a defender is rushing the quarterback (these players were identified and removed prior to clustering).

![](https://raw.githubusercontent.com/jvenzor23/DefensiveCoverageNet/master/src/08_writeup/images/CoverageEx3.gif)

The play animated on the left above illustrates the tagging of a Cover 2 Man defensive play call, and the play on the right above illustrates the tagging of a Cover 3 defensive play call (note that #20 is still identified as playing man coverage).

### A.2 EPA Model Player Tracking Features <a id="A.2"></a>

![](https://raw.githubusercontent.com/jvenzor23/DefensiveCoverageNet/master/src/08_writeup/images/CNNFeauturesTable2.png)

### A.3 Tracking EPAs Examples <a id="A.3"></a>

__Positive Tracking Example__: (Marlon Humphery, bottom sideline)
![](https://raw.githubusercontent.com/jvenzor23/DefensiveCoverageNet/master/src/08_writeup/images/TrackingGoodEx2.gif)

__Negative Tracking Example__: (Robert Alford, Slot Corner)
![](https://raw.githubusercontent.com/jvenzor23/DefensiveCoverageNet/master/src/08_writeup/images/TrackingBadEx2.gif)

### A.4 Closing EPAs Examples <a id="A.4"></a>

__Positive Closing Example__ (D.J. Hayden, Slot Corner) and __Negative Example__ (Xavier Rhodes)
<table>
<tr>
    <td> <img src="https://raw.githubusercontent.com/jvenzor23/DefensiveCoverageNet/master/src/08_writeup/NFL_videos/closing_good.gif" alt="Drawing" style="width: 300px;"/> </td>
    <td> <img src="https://raw.githubusercontent.com/jvenzor23/DefensiveCoverageNet/master/src/08_writeup/intermediates/closing_good_intermediate.png" alt="Drawing" style="width: 500px;"/> </td>
    </tr>
   </table>
   
 <table>
 <tr>
    <td> <img src="https://raw.githubusercontent.com/jvenzor23/DefensiveCoverageNet/master/src/08_writeup/NFL_videos/closing_bad.gif" alt="Drawing" style="width: 300px;"/> </td>
    <td> <img src="https://raw.githubusercontent.com/jvenzor23/DefensiveCoverageNet/master/src/08_writeup/intermediates/closing_bad_intermediate.png" alt="Drawing" style="width: 500px;"/> </td>
    </tr>
    
   </table>

### A.5 Ball Skills EPAs Examples <a id="A.5"></a>

__Positive Ball Skills Example__ (Xavien Howard) and __Negative Example__ (Donte Jackson)
 <table>
 <tr>
    <td> <img src="https://raw.githubusercontent.com/jvenzor23/DefensiveCoverageNet/master/src/08_writeup/NFL_videos/ball_skills_good.gif" alt="Drawing" style="width: 300px;"/> </td>
    <td> <img src="https://raw.githubusercontent.com/jvenzor23/DefensiveCoverageNet/master/src/08_writeup/intermediates/ball_skills_good_intermediate.png" alt="Drawing" style="width: 500px;"/> </td>
    </tr>
 </table>
   
 <table>
 <tr>
    <td> <img src="https://raw.githubusercontent.com/jvenzor23/DefensiveCoverageNet/master/src/08_writeup/NFL_videos/ball_skills_bad.gif" alt="Drawing" style="width: 300px;"/> </td>
    <td> <img src="https://raw.githubusercontent.com/jvenzor23/DefensiveCoverageNet/master/src/08_writeup/intermediates/ball_skills_bad_intermediate.png" alt="Drawing" style="width: 500px;"/> </td>
    </tr>
    </table>

### A.6 Positive Tackling Example (Jalen Ramsey) <a id="A.6"></a>

<table>
 <tr>
    <td> <img src="https://raw.githubusercontent.com/jvenzor23/DefensiveCoverageNet/master/src/08_writeup/NFL_videos/tackling_good.gif" alt="Drawing" style="width: 300px;"/> </td>
    <td> <img src="https://raw.githubusercontent.com/jvenzor23/DefensiveCoverageNet/master/src/08_writeup/intermediates/tackle_good_intermediate.png" alt="Drawing" style="width: 500px;"/> </td>
    </tr>
</table>

### A.7 Ball Hawk Example (Denzel Ward, top of screen) <a id="A.7"></a>

<table>
 <tr>
    <td> <img src="https://raw.githubusercontent.com/jvenzor23/DefensiveCoverageNet/master/src/08_writeup/NFL_videos/ball_hawk.gif" alt="Drawing" style="width: 300px;"/> </td>
    <td> <img src="https://raw.githubusercontent.com/jvenzor23/DefensiveCoverageNet/master/src/08_writeup/intermediates/ball_hawk_intermediate.png" alt="Drawing" style="width: 500px;"/> </td>
    </tr>
</table>